# Capturing Sentimental Opinions on Presidential Candidates

In this project, we'll aim to capture the opinions and sentiments of a sample of electoral voters. We'll do this by analyzing their tweets on a daily basis over a period of time to see how these opinions are varying or changing over time. 

Can analyzing sentiments of tweets from a sample during a given period provide insight to how an election will swing? The main goal of the project is to examine and analyze tweets containing reference to candidates vying for a presidential spot, classify these tweets into positive, negative or neutral sentiments and from these, try to conclude on how much online sentiments provide for a good indicator to predict the results of an election.

## Training Data Set 

The data set we used in this project is the Sentiment140 dataset. You can find the data set [here](http://help.sentiment140.com/for-students). 

## Exploring the Data Set 

Below we'll do a quick exploration of the data set to find irregularities and understand the composition of the data.

### What questions should we try to answer while looking at the data? (Please add ideas)
1. How many positive/negative records do we have?
2. Are there null/empty records?
3. Do we really need all the columns? Which features are important?
4. The text column, do we need @usernames, RTs, and any unnecessary text?
5. Are we taking neutral records into consideration or just positives and negatives?
6. Do we take into consideration tweets that contain e.g "I wish", "I hope" (without comparison these tweets might be deemed neutral, otherwise positive or negative).
7. How do we plan to prepare data to prevent bias? (i.e Ratio of positive to negative tweets), to prevent the model from being biased towards the seniment with the higher ratio?

In [39]:
import pandas as pd
import numpy as np
dataset = "./dataset/training.1600000.processed.noemoticon.csv"
cols = ['sentiment', 'id', 'date', 'query', 'username', 'text']
df = pd.read_csv(dataset, header=None, names=cols, encoding='latin-1')
df.head(10)

,sentiment,id,date,query,username,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [40]:
df.describe()

,sentiment,id
count,1.600000e+06,1.600000e+06
mean,2.000000e+00,1.998818e+09
std,2.000001e+00,1.935761e+08
min,0.000000e+00,1.467810e+09
25%,0.000000e+00,1.956916e+09
50%,2.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


### Question 1:
There are 1.6 million records and 6 columns

In [41]:
df.shape

(1600000, 6)

In [42]:
df.index

RangeIndex(start=0, stop=1600000, step=1)

In [43]:
df.columns

Index(['sentiment', 'id', 'date', 'query', 'username', 'text'], dtype='object')

### Question 2:
There are no null values in the dataset

In [44]:
df.isnull().sum()

sentiment    0
id           0
date         0
query        0
username     0
text         0
dtype: int64

### Question 3:
The columns id, date, query and username will not be useful in the training of the data so we will get rid of them

In [45]:
del df['id']

In [46]:
del df['date']

In [47]:
del df['query']

In [48]:
del df['username']

In [49]:
df.columns

Index(['sentiment', 'text'], dtype='object')

### Question 4:
Cleaning up the text column

### Question 5 and 7:
The dataset we will use for the training doesn't have neutral sentiments, only (0 - negative and 4 - positive) So we will just work with positives and negatives. Since the dataset is balanced between the two classes (800,000), bias shouldn't be a problem

In [50]:
df['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

### Question 6:
True without further context, tweets with those terms will be deemde neutral but the model has to take the whole context of the sentence/tweet into consideration before classifying into positive or negative.
E.g. I hope Buhari doesn't become president again = Negative
     I wish I had been more accepting of Atiku = Positive